# 0. 구글 드라이브와 연결 및 각종 라이브러리 설정

### 구글 드라이브와 연결

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 라이브러리 설치

In [2]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 510 kB/s 
     |████████████████████████████████| 453 kB 67.9 MB/s 


### 라이브러리 불러오기

In [4]:
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import re

In [5]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Unzipping grammars/book_grammars.zip.
       | Downloading 


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

# 1. 데이터 로드

### 데이터 로드

In [6]:
train0 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/감성대화말뭉치(최종데이터)_Training.xlsx', engine='openpyxl')

In [7]:
validation0 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/감성대화말뭉치(최종데이터)_Validation.xlsx', engine='openpyxl')

In [8]:
nsmc0 = pd.read_csv('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/nsmc_data.txt', sep='\t')

### 가져온 데이터 확인

In [9]:
train0.head(2)

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN


In [10]:
validation0.head(2)

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,23093,청년,여성,"진로, 취업, 직장",해당없음,분노,툴툴대는,요즘 부모님과 많이 부딪혀.,어떤 일로 부딪히시나요?,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...,부모님과의 충돌로 힘드시겠군요.,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.,자신이 하고 싶은 일을 목표로 하기로 하셨군요.,NaN,NaN
1,32848,청소년,남성,가족관계,해당없음,슬픔,비통한,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,어머니께서 집을 나가셨다니 정말 슬프셨겠어요. 좀 더 자세히 얘기해주시겠어요?,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...,정말 많이 답답하셨겠어요. 어떻게 하는 게 기분이 좀 풀릴까요?,일단 좀 자야겠어.,일단 휴식을 가질 생각이시군요.,NaN,NaN


In [11]:
nsmc0.head(2)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1


### sample의 개수 확인

In [12]:
print('train0의 sample 수: {}개'.format(train0.shape[0]))
print('validation0의 sample 수: {}개'.format(validation0.shape[0]))
print('nsmc0 sample 수: {}개'.format(nsmc0.shape[0]))
print('모든 샘플의 수: {}개'.format(train0.shape[0]+validation0.shape[0]+nsmc0.shape[0]))

train0의 sample 수: 40879개
validation0의 sample 수: 5130개
nsmc0 sample 수: 200000개
모든 샘플의 수: 246009개


# 2. 데이터 정제
### 중복된 값, 결측 값

### nsmc 데이터 확인 및 정제

`결측값 제거`

In [13]:
# 결측값을 확인

nsmc0.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [14]:
nsmc0.loc[nsmc0['document'].isnull()]

,id,document,label
46471,6369843,NaN,1
60735,511097,NaN,1
77665,2172111,NaN,1
84098,402110,NaN,1
127017,5942978,NaN,0
172375,5026896,NaN,0
173526,1034280,NaN,0
197279,1034283,NaN,0


In [15]:
nsmc1 = nsmc0.dropna(how='any')

In [16]:
# 결측값을 확인

nsmc1.isnull().sum()

id          0
document    0
label       0
dtype: int64

`중복값 제거`

In [17]:
print('nsmc1 데이터 샘플의 수: {}'.format(nsmc1.shape[0]))
print('nsmc1 데이터의 document 종류의 수: {}'.format(nsmc1['document'].nunique()))

nsmc1 데이터 샘플의 수: 199992
nsmc1 데이터의 document 종류의 수: 194543


In [18]:
nsmc2 = nsmc1.drop_duplicates(subset=['document'], ignore_index=True)

In [19]:
print('nsmc2 데이터 샘플의 수: {}'.format(nsmc2.shape[0]))
print('nsmc2 데이터의 document 종류의 수: {}'.format(nsmc2['document'].nunique()))

nsmc2 데이터 샘플의 수: 194543
nsmc2 데이터의 document 종류의 수: 194543


`노이즈 제거`

In [20]:
f = lambda sent: 'ㅋ' in sent
f('안녕 ')

False

In [21]:
'ㅋ' in '캐꿀잼'

False

In [22]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅎ' in sent)]

,id,document,label
49,10272889,내용이너무좋아요.ㅎ,1
68,10092383,"우리애기첫영화 너무너무 잘선택한거같아요""""ㅎ굳",1
90,9097219,어설픈건 사실이었다 하지 재밋었다 ㅎ,1
111,6906147,ㅎㅎㅎㅎㅎㅎㅎㅎ,1
113,9484605,정말행복했다... 계속미소지으면서봐서 보조개가아프네 ㅎㅎ,1
...,...,...,...
193832,9149546,저 영화보다가 포기한적 처음이예요 ㅎㅎ,0
194142,9027137,어떨까 하고 봤는데 개장수 배우ㅋㅎ.,0
194308,2516602,그저그랬어요. 후카다 쿄코는 한마디도 없이 웃기만 하네 ㅎ,0
194349,10238609,이제 b 29 에놀라가 히로시마 상공에서 리틀보이를 사랑으로 떨어뜨리는 영화를만들면...,0


In [23]:
# 정규표현식을 사용하여 문장 내 자음만 있는 것을 찾아서 어떤 것들이 있나 확인해보자

In [24]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅋ' in sent)]['label'].value_counts()

1    7199
0    5406
Name: label, dtype: int64

In [25]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅎ' in sent)]['label'].value_counts()

1    3064
0     696
Name: label, dtype: int64

In [26]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅜ' in sent)]['label'].value_counts()

1    1330
0     717
Name: label, dtype: int64

In [27]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅠ' in sent)]['label'].value_counts()

1    3628
0    1509
Name: label, dtype: int64

In [28]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㄷ' in sent)]['label'].value_counts()

1    501
0    308
Name: label, dtype: int64

In [29]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'b' in sent)]['label'].value_counts()

1    577
0    398
Name: label, dtype: int64

In [30]:
p = re.compile('[ㄱ-ㅎㅏ-ㅣ]')

In [31]:
result = p.findall('안녕하세요 ㅜㅜ 저는 오창준입니다 ㅎㅎ')

In [32]:
result

['ㅜ', 'ㅜ', 'ㅎ', 'ㅎ']

In [33]:
p = re.compile('[ㄱ-ㅎㅏ-ㅣ]')

In [34]:
def find_con(sent):
  result = p.findall(sent)
  return set(result)

In [35]:
find_con('안녕하세요 ㅜㅜ 저는 오창준입니다 ㅎㅎ')

{'ㅎ', 'ㅜ'}

In [36]:
 nsmc2

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [37]:
nsmc3 = nsmc2.copy()

In [38]:
nsmc3['con'] = 0

In [39]:
nsmc2

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [40]:
nsmc3['con'] = nsmc3['document']

In [41]:
nsmc3

,id,document,label,con
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,어릴때보고 지금다시봐도 재밌어요ㅋㅋ
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산..."
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
...,...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0,포켓 몬스터 짜가 ㅡㅡ;;
194539,3302770,쓰.레.기,0,쓰.레.기
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ


In [42]:
nsmc3['con'] = nsmc3['con'].map(find_con)

In [43]:
nsmc3

,id,document,label,con
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,{ㅋ}
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,{}
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,{}
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,{}
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,{}
...,...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0,{ㅡ}
194539,3302770,쓰.레.기,0,{}
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0,{}
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0,"{ㅠ, ㅋ}"
